In [3]:
"""
LEGEND200 Detector Mapper Tool
Author: Marta Babicz
Date: January 2025
Description:
This script provides an interactive tool for visualizing and filtering HPGe detectors 
in the LEGEND200 experiment. Users can filter detectors based on usability (active, AC, or off),
select specific periods and runs, and visualize detector positions in an interactive plot.

Dependencies:
- legend200_data_loader
- legendmeta
- ipywidgets
- matplotlib
"""

# ======================== IMPORTS ========================
import datetime as dt
import awkward as ak
import pandas as pd
import pickle
import re
import h5py
import os

from legend200_data_loader.loader import LegendDataLoader
from legend200_data_loader.preprocessing import plot_detector_positions
from legend200_data_loader.function import filter_metadata
from legendmeta import LegendMetadata
from ipywidgets import interact, widgets
from IPython.display import display

In [4]:
# ======================== SSH SETUP ========================
# Set up SSH agent for authentication (before in terminal)!
os.environ["SSH_AUTH_SOCK"] = "/tmp/ssh-XXXXXX1U2ma8/agent.2354076"

In [5]:
# ======================== LOAD METADATA ========================
# Initialize LegendMetadata for accessing detector metadata
lmeta = LegendMetadata()

# Step 1: Metadata Filtering (e.g., 'geds')
detector_type = 'geds'  
detectors = filter_metadata(lmeta)


if not detectors:
    print(f"No detectors found for type: {detector_type}")
else:
    detector_entries = {key: value for key, value in detectors.items() if key != 'rawid_lists'}
    print(f"Found {len(detector_entries)} HPGe detectors.")

Found 101 HPGe detectors.


In [6]:
# ======================== LOAD DATA LOADER ========================
# Initialize the LegendDataLoader for accessing files and runs
data_loader = LegendDataLoader()

Dropdown(description='Period:', options=('p08/', 'p09/', 'p11/'), value='p08/')

Dropdown(description='Run:', options=(), value=None)

Checkbox(value=True, description='Show Active Detectors (on)')

Checkbox(value=True, description='Show AC Detectors (ac)')

Checkbox(value=False, description='Show Off Detectors (off)')

In [9]:
# ======================== INTERACTIVE WIDGETS ========================
#Dropdown for selecting periods
available_periods = ['p08/', 'p09/', 'p11/']  # Replace with actual periods
period_dropdown = widgets.Dropdown(
    options=available_periods,
    value=available_periods[0],
    description='Period:'
)

#Dropdown for selecting runs
run_dropdown = widgets.Dropdown(
    options=[],  # Updated dynamically based on selected period
    description='Run:'
)


show_active = widgets.Checkbox(value=True, description='Show Active Detectors (on)')
show_ac = widgets.Checkbox(value=True, description='Show AC Detectors (ac)')
show_off = widgets.Checkbox(value=False, description='Show Off Detectors (off)')

# Display widgets
display(period_dropdown, run_dropdown, show_active, show_ac, show_off)

# ======================== UPDATE FUNCTIONS ========================
# Function to update runs based on the selected period
def update_runs(period):
    runs = data_loader.get_runs(period)
    run_dropdown.options = runs
    if runs:
        run_dropdown.value = runs[0]

# Observe changes in the period dropdown and update runs dynamically
period_dropdown.observe(lambda change: update_runs(change.new), names='value')
update_runs(period_dropdown.value)

# ======================== SAVE PLOT OPTIONS ========================
#You can checkbox and text box for saving plots
save_plot_checkbox = widgets.Checkbox(value=False, description='Save Plot')
save_path_text = widgets.Text(
    value='/global/u2/m/mababicz/l200-detector-mapper/detector_plot.png', #but change the path!
    placeholder='Enter save path',
    description='Save Path:'
)

# Display save options widgets
display(save_plot_checkbox, save_path_text)

# ======================== PLOT FUNCTION ========================

def update_plot(period, run, active, ac, off, save_plot, save_path):
    """
    Update and visualize detector positions based on user inputs.
    """
    #Load files for the selected period and run
    file_dict = data_loader.load_files(period, run)
    tier_to_check = 'raw_files'

    
    timestamps = []
    for file_path in file_dict.get(tier_to_check, []):
        filename = os.path.basename(file_path)
        match = re.search(r'\d{8}T\d{6}Z', filename)
        if match:
            timestamps.append(match.group())

    if not timestamps:
        print(f"No valid timestamps found in {tier_to_check} files.")
        return

    
    timestamp = timestamps[0]
    channel_map = lmeta.channelmap(timestamp)

    
    usable_detectors, ac_detectors, off_detectors = [], [], []
    for det_id in detectors:
        detector_info = channel_map.get(det_id, {})
        analysis_info = detector_info.get('analysis', {})
        usability = analysis_info.get('usability', 'unknown')

        if usability == 'on' and active:
            usable_detectors.append(det_id)
        elif usability == 'ac' and ac:
            ac_detectors.append(det_id)
        elif usability == 'off' and off:
            off_detectors.append(det_id)

   
    save_path_actual = save_path if save_plot else None
    plot_detector_positions(
        usable_detectors,
        lmeta,
        period,
        run,
        ac_detectors=ac_detectors,
        off_detectors=off_detectors,
    )
    if save_plot:
        print(f"Plot saved to {save_path}")

# ======================== INTERACTIVE PLOT ========================

interact(
    update_plot,
    period=period_dropdown,
    run=run_dropdown,
    active=show_active,
    ac=show_ac,
    off=show_off,
    save_plot=save_plot_checkbox,
    save_path=save_path_text
)


Dropdown(description='Period:', options=('p08/', 'p09/', 'p11/'), value='p08/')

Dropdown(description='Run:', options=(), value=None)

Checkbox(value=True, description='Show Active Detectors (on)')

Checkbox(value=True, description='Show AC Detectors (ac)')

Checkbox(value=False, description='Show Off Detectors (off)')

Checkbox(value=False, description='Save Plot')

Text(value='/global/u2/m/mababicz/l200-detector-mapper/detector_plot.png', description='Save Path:', placehold…

interactive(children=(Dropdown(description='Period:', options=('p08/', 'p09/', 'p11/'), value='p08/'), Dropdow…

<function __main__.update_plot(period, run, active, ac, off, save_plot, save_path)>